# Capstone Project - The Battle of Neighbourhoods: Should I move from Lausanne to Bern?
Christopher Tse, Dec 2019
Compare Lausanne and Bern, and using data science techniques determine which neighbourhood would be best for me and explain why.

## Introduction
I am a highly qualified biotech scientist living in Switzerland. I'm currently living in a city called Lausanne, on the French-speaking side of Switzerland, but there may be more job and life opportunities in a German-speaking city called Bern in Switzerland. Bern is very popular among consultancy, biotech and pharmaceutical companies and I want to use data science to help me decide which city would suit me best to live.

I want to compare the livability of the two cities through using similar methods that have been used in the assignments from this course, and help me decide which area would be best for me - a young adult. This project would also be of interest for other individuals individuals and organisations who are interested in assessing these two cities for business and living. 

## Data

In this project, I will use Foursquare data to compare the most interesting regions of Lausanne and Bern. This will be done with k-means to segment and cluster related neighbourhoods to allow me to estimate the suitable demographic to live there. I will be interested in areas which are near a supermarket for convenience, and survey the types of venues in the area.
The use of folium interactive maps would be very useful here also so I can visually see attractive features like rivers and parks in the respective areas to strengthen the validity of my recommendations.

There is a limit to how many venues each query can retrieve (up to 50), therefore I will need to break up my query into several queries to cover as much of each city as possible. 
A local governmnet-run supermarket chain called Migros are well distributed across all cities to provide the sale of groceries to the local populations. I will use these as local "neighbourhoods" to capture as many venues as possible. 

## Summary of Work
Using Foursquare, I first start at the train station from each city. From the train station, I select all Migros supermarkets nearby. From each Migros supermarket, I list out all nearby venues and analyse each of these "neighbourhoods". 


## Methodology
First I will load up all the libraries which I think I will use.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Define Foursquare credentials and version

In [2]:
CLIENT_ID = 'OHYIJKDCMRFM0YTYCZRJHC5H2I5ZA51HU41EU5LK1W5PVAPS' # your Foursquare ID
CLIENT_SECRET = 'G2KF03HSYHCLVCHN11ZBGR5GZDVKBYOG2UVF20X4BPQLQ15G' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OHYIJKDCMRFM0YTYCZRJHC5H2I5ZA51HU41EU5LK1W5PVAPS
CLIENT_SECRET:G2KF03HSYHCLVCHN11ZBGR5GZDVKBYOG2UVF20X4BPQLQ15G


### For each city, I will start with the train station, and from there, select all nearby Migros supermarkets.

In [3]:
address = 'Place de la Gare 5A, 1003 Lausanne'
# For my work in Bern, I will find '3008 Bern'
# Place de la Gare 5A, 1003 Lausanne
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
print(location)

46.516758 6.6301341
5a, Place de la Gare, Lausanne, District de Lausanne, Vaud, 1003, Switzerland


### Having the location data, I can now use the Foursquare API to find all the Migros venues around the station. I set a radius of 10000 (10km) and set the limit to the maximum of 50 venues. 

In [4]:
search_query = 'Migros'
radius = 10000
print(search_query + ' .... OK!')

Migros .... OK!


### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OHYIJKDCMRFM0YTYCZRJHC5H2I5ZA51HU41EU5LK1W5PVAPS&client_secret=G2KF03HSYHCLVCHN11ZBGR5GZDVKBYOG2UVF20X4BPQLQ15G&ll=46.516758,6.6301341&v=20180604&query=Migros&radius=10000&limit=50'

### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df6370d9388d7001bf3a68b'},
 'response': {'venues': [{'id': '4bd2e821b221c9b6ac6fd8d0',
    'name': 'Migros',
    'location': {'address': "Avenue d'Ouchy 11",
     'lat': 46.51503921361126,
     'lng': 6.63274963922186,
     'labeledLatLngs': [{'label': 'display',
       'lat': 46.51503921361126,
       'lng': 6.63274963922186}],
     'distance': 277,
     'postalCode': '1006',
     'cc': 'CH',
     'city': 'Lausanne',
     'state': 'Vaud',
     'country': 'Suisse',
     'formattedAddress': ["Avenue d'Ouchy 11", '1006 Lausanne', 'Suisse']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b46',
      'name': 'Supermarket',
      'pluralName': 'Supermarkets',
      'shortName': 'Supermarket',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1576417067',
    'hasPerk': False},
   {'id': '4cc1bc9e43646dcb16ab478c',
    'name': 'Migros',
    'loc

### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(50)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4bd2e821b221c9b6ac6fd8d0,Avenue d'Ouchy 11,CH,Lausanne,Suisse,NaN,277,"[Avenue d'Ouchy 11, 1006 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.51503921361126...",46.515039,6.632750,1006,Vaud,Migros,v-1576417067,NaN
1,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4cc1bc9e43646dcb16ab478c,Rue des Terreaux 23,CH,Lausanne,Suisse,NaN,668,"[Rue des Terreaux 23, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52248959188916...",46.522490,6.627551,1003,Vaud,Migros,v-1576417067,NaN
2,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4bc1e78fb492d13a8f94a660,Avenue de Rhodanie 2,CH,Lausanne,Suisse,NaN,1081,"[Avenue de Rhodanie 2, 1007 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.50773468857085...",46.507735,6.624897,1007,Vaud,Migros,v-1576417067,NaN
3,"[{'id': '4bf58dd8d48988d1a8941735', 'name': 'G...",False,4c7bd61497028cfa4600e2fe,Rue de Genève 35,CH,Lausanne,Suisse,NaN,707,"[Rue de Genève 35, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52220032051376...",46.522200,6.625372,1003,Vaud,Ecole Club Migros,v-1576417067,NaN
4,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4b704670f964a5205e0f2de3,Voie du Chariot 4-6,CH,Lausanne,Suisse,NaN,480,"[Voie du Chariot 4-6, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52100915635693...",46.521009,6.629060,1003,Vaud,Migros,v-1576417067,NaN
5,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4b56eab0f964a5202c1f28e3,Ave. de Sévelin 2A,CH,Lausanne,Suisse,NaN,1197,"[Ave. de Sévelin 2A, 1007 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52135186258868...",46.521352,6.615997,1007,Vaud,Migros,v-1576417067,NaN
6,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4bde7292e75c0f478ff2c703,Rue du Maupas 63,CH,Lausanne,Suisse,NaN,1146,"[Rue du Maupas 63, 1004 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52600298499138...",46.526003,6.623536,1004,Vaud,Migros,v-1576417067,NaN
7,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,4bf987108d30d13a6a610218,Avenue de la Vallonnette 2,CH,Lausanne,Suisse,NaN,2004,"[Avenue de la Vallonnette 2, 1012 Lausanne, Su...","[{'label': 'display', 'lat': 46.52705916235473...",46.527059,6.651592,1012,Vaud,Migros,v-1576417067,NaN
8,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4b5071b2f964a520742327e3,Ch. du Pré-de-la-Tour 3,CH,Pully,Suisse,NaN,2333,"[Ch. du Pré-de-la-Tour 3, 1009 Pully, Suisse]","[{'label': 'display', 'lat': 46.51126962847088...",46.511270,6.659531,1009,Vaud,Migros,v-1576417067,NaN
9,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4d56aab958856dcb44de5a6d,Place de la Croix-Blanche 11,CH,Epalinges,Suisse,NaN,4690,"[Place de la Croix-Blanche 11, 1066 Epalinges,...","[{'label': 'display', 'lat': 46.54840656841466...",46.548407,6.670572,1066,Vaud,Migros,v-1576417067,NaN


### Define information of interest and filter dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Migros,Supermarket,Avenue d'Ouchy 11,CH,Lausanne,Suisse,NaN,277,"[Avenue d'Ouchy 11, 1006 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.51503921361126...",46.515039,6.632750,1006,Vaud,4bd2e821b221c9b6ac6fd8d0
1,Migros,Supermarket,Rue des Terreaux 23,CH,Lausanne,Suisse,NaN,668,"[Rue des Terreaux 23, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52248959188916...",46.522490,6.627551,1003,Vaud,4cc1bc9e43646dcb16ab478c
2,Migros,Grocery Store,Avenue de Rhodanie 2,CH,Lausanne,Suisse,NaN,1081,"[Avenue de Rhodanie 2, 1007 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.50773468857085...",46.507735,6.624897,1007,Vaud,4bc1e78fb492d13a8f94a660
3,Ecole Club Migros,General College & University,Rue de Genève 35,CH,Lausanne,Suisse,NaN,707,"[Rue de Genève 35, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52220032051376...",46.522200,6.625372,1003,Vaud,4c7bd61497028cfa4600e2fe
4,Migros,Grocery Store,Voie du Chariot 4-6,CH,Lausanne,Suisse,NaN,480,"[Voie du Chariot 4-6, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52100915635693...",46.521009,6.629060,1003,Vaud,4b704670f964a5205e0f2de3
5,Migros,Grocery Store,Ave. de Sévelin 2A,CH,Lausanne,Suisse,NaN,1197,"[Ave. de Sévelin 2A, 1007 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52135186258868...",46.521352,6.615997,1007,Vaud,4b56eab0f964a5202c1f28e3
6,Migros,Grocery Store,Rue du Maupas 63,CH,Lausanne,Suisse,NaN,1146,"[Rue du Maupas 63, 1004 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52600298499138...",46.526003,6.623536,1004,Vaud,4bde7292e75c0f478ff2c703
7,Migros,Department Store,Avenue de la Vallonnette 2,CH,Lausanne,Suisse,NaN,2004,"[Avenue de la Vallonnette 2, 1012 Lausanne, Su...","[{'label': 'display', 'lat': 46.52705916235473...",46.527059,6.651592,1012,Vaud,4bf987108d30d13a6a610218
8,Migros,Grocery Store,Ch. du Pré-de-la-Tour 3,CH,Pully,Suisse,NaN,2333,"[Ch. du Pré-de-la-Tour 3, 1009 Pully, Suisse]","[{'label': 'display', 'lat': 46.51126962847088...",46.511270,6.659531,1009,Vaud,4b5071b2f964a520742327e3
9,Migros,Grocery Store,Place de la Croix-Blanche 11,CH,Epalinges,Suisse,NaN,4690,"[Place de la Croix-Blanche 11, 1066 Epalinges,...","[{'label': 'display', 'lat': 46.54840656841466...",46.548407,6.670572,1066,Vaud,4d56aab958856dcb44de5a6d


### Let's visualize the venues that are in Lausanne, with a focus from the train station

In [9]:
dataframe_filtered.name

0                                             Migros
1                                             Migros
2                                             Migros
3                                  Ecole Club Migros
4                                             Migros
5                                             Migros
6                                             Migros
7                                             Migros
8                                             Migros
9                                             Migros
10                                            Migros
11                                     Pop Up Migros
12                                    Migros de Cour
13                                            Migros
14                                            Migros
15                                            Migros
16                          Migros M Prilly - Malley
17                                  Migros Take Away
18                                            

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the train sation
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Lausanne Gare',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the venues as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Let's create a function to repeat the same process to all the areas near a Mirgos in Lausanne

In [11]:
#
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now we write the code to run the above function on each neighborhood and create a new dataframe called *lausanne_areas*, with the unique ID of each Migros store

In [12]:
lausanne_areas = getNearbyVenues(names=dataframe_filtered['id'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )

4bd2e821b221c9b6ac6fd8d0
4cc1bc9e43646dcb16ab478c
4bc1e78fb492d13a8f94a660
4c7bd61497028cfa4600e2fe
4b704670f964a5205e0f2de3
4b56eab0f964a5202c1f28e3
4bde7292e75c0f478ff2c703
4bf987108d30d13a6a610218
4b5071b2f964a520742327e3
4d56aab958856dcb44de5a6d
4b72a011f964a5200b812de3
5d97b0c2112828000771ae34
4d761b3774eca093a8e8b6a8
4eca59250e6158ba01dc50c8
4b6a9c2ff964a5201ada2be3
4e68c32e52b128a09edb7b50
4d42a6fa714a224bea77e5e8
4bc59590ccbcef3befaae6d2
4bdb17f22a3a0f472461adb6
4cd2b48f158c54816d6db0e4
4e8eabe95c5c312991253278
5094e1eae4b0c4415627e9c5
4cb5c8c752edb1f7c8cb69fe
4c29f6fe3492a59381adb928
4c790e083badb1f7b89e4d54
4d2c3c11f7a9224bcb2e16a0
4e381063c65bf39cbb3481c2
4fd0c66ce4b01e9f634e5347
4e732c437d8b051837d090f3
4f708ff5e4b0e0075d557ea6
4b5db279f964a520566729e3
523a937711d21ca5d858ed0d
5229aca411d23309d4d37ec0
5034050fe4b0913039fbdd8c
5992bce3f870fd22a1ed1887
520b9d3411d28799c60a4f97
4c3f1c9f6faac9b6291b0f76
4e0f364e1838f62321cd0f41
5c4c75e225ecca002cc9291c
4f757067e4b0bf1519a480ad


### Let's check the size of the resulting dataframe

In [13]:
print(lausanne_areas.shape)
lausanne_areas.head()

(1616, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4bd2e821b221c9b6ac6fd8d0,46.515039,6.63275,Sleepy Bear Coffee,46.515505,6.631326,Coffee Shop
1,4bd2e821b221c9b6ac6fd8d0,46.515039,6.63275,Les Trois Rois,46.515515,6.631223,French Restaurant
2,4bd2e821b221c9b6ac6fd8d0,46.515039,6.63275,Café du Simplon,46.515961,6.629948,Mediterranean Restaurant
3,4bd2e821b221c9b6ac6fd8d0,46.515039,6.63275,Pasta e Sfizi,46.516374,6.633477,Italian Restaurant
4,4bd2e821b221c9b6ac6fd8d0,46.515039,6.63275,Les Gosses du Québec,46.517072,6.633122,Bar


In [14]:
print('There are {} uniques categories.'.format(len(lausanne_areas['Venue Category'].unique())))

There are 142 uniques categories.


In [15]:
# one hot encoding
lausanne_onehot = pd.get_dummies(lausanne_areas[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lausanne_onehot['Neighborhood'] = lausanne_areas['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lausanne_onehot.columns[-1]] + list(lausanne_onehot.columns[:-1])
lausanne_onehot = lausanne_onehot[fixed_columns]

lausanne_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Bookstore,Botanical Garden,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Construction & Landscaping,Convenience Store,Creperie,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Lake,Light Rail Station,Lounge,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Multiplex,Museum,Music Store,Nightclub,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Rest Area,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Snack Place,Soccer Field,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Wine Bar,Winery,Zoo Exhibit
0,4bd2e821b221c9b6ac6fd8d0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4bd2e821b221c9b6ac6fd8d0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4bd2e821b221c9b6ac6fd8d0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4bd2e821b221c9b6ac6fd8d0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4bd2e821b221c9b6ac6fd8d0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Let's check how many venues were returned for each neighborhood

In [16]:
lausanne_onehot.shape

(1616, 143)

In [17]:
lausanne_grouped = lausanne_onehot.groupby('Neighborhood').mean().reset_index()
lausanne_grouped

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Bookstore,Botanical Garden,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Construction & Landscaping,Convenience Store,Creperie,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Lake,Light Rail Station,Lounge,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Multiplex,Museum,Music Store,Nightclub,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Rest Area,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Snack Place,Soccer Field,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Wine Bar,Winery,Zoo Exhibit
0,4b5071b2f964a520742327e3,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.050000,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.05,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150000,0.000000,0.000000,0.050000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.05,0.000000,0.000000,0.050000,0.050000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,4b56eab0f964a5202c1f28e3,0.02,0.020000,0.02,0.020000,0.040000,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.00,0.040000,0.00,0.020000,0.000000,0.000000,0.00,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.020000,0.020000,0.060000,0.060000,0.020000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.040000,0.000000,0.000000,

### Let's print each neighborhood along with the top 5 most common venues

In [18]:
num_top_venues = 5

for hood in lausanne_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = lausanne_grouped[lausanne_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----4b5071b2f964a520742327e3----
              venue  freq
0              Park  0.15
1              Pool  0.10
2     Grocery Store  0.05
3  Swiss Restaurant  0.05
4    Sandwich Place  0.05


----4b56eab0f964a5202c1f28e3----
                 venue  freq
0                  Bar  0.06
1   Italian Restaurant  0.06
2  Japanese Restaurant  0.06
3                 Park  0.06
4               Lounge  0.04


----4b5db279f964a520566729e3----
                  venue  freq
0           Supermarket  0.12
1            Restaurant  0.06
2         Train Station  0.06
3              Bus Stop  0.06
4  Gym / Fitness Center  0.06


----4b6a9c2ff964a5201ada2be3----
                venue  freq
0    Swiss Restaurant  0.10
1              Bakery  0.10
2                Park  0.10
3         Zoo Exhibit  0.05
4  Italian Restaurant  0.05


----4b704670f964a5205e0f2de3----
               venue  freq
0                Bar  0.12
1  French Restaurant  0.06
2              Plaza  0.06
3        Pizza Place  0.06
4       Burger

           venue  freq
0    Pizza Place  0.50
1   Home Service  0.25
2  Train Station  0.25
3      Pet Store  0.00
4    Music Store  0.00


----520b9d3411d28799c60a4f97----
                venue  freq
0                 Gym  0.10
1         Supermarket  0.10
2          Restaurant  0.07
3   Electronics Store  0.07
4  Italian Restaurant  0.07


----5229aca411d23309d4d37ec0----
              venue  freq
0       Supermarket  0.18
1  Sushi Restaurant  0.12
2            Bakery  0.12
3             Hotel  0.06
4            Museum  0.06


----523a937711d21ca5d858ed0d----
                venue  freq
0                Café  0.14
1               Hotel  0.10
2  Light Rail Station  0.10
3      Science Museum  0.05
4        Burger Joint  0.05


----52a32bda11d26c8eeb056266----
                    venue  freq
0               Pet Store  0.12
1  Furniture / Home Store  0.12
2        Department Store  0.12
3          Clothing Store  0.12
4           Shopping Mall  0.12


----5992bce3f870fd22a1ed1887----
   

### Let's put that into a *pandas* dataframe

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lausanne_grouped['Neighborhood']

for ind in np.arange(lausanne_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lausanne_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4b5071b2f964a520742327e3,Park,Pool,Indian Restaurant,General Entertainment,Sandwich Place,Bus Station,Coffee Shop,Supermarket,Sushi Restaurant,Swiss Restaurant
1,4b56eab0f964a5202c1f28e3,Japanese Restaurant,Park,Bar,Italian Restaurant,Asian Restaurant,Bistro,Burger Joint,Nightclub,Lounge,Sushi Restaurant
2,4b5db279f964a520566729e3,Supermarket,Bus Stop,Gym / Fitness Center,Restaurant,Electronics Store,Train Station,Gym,Bowling Alley,Furniture / Home Store,Park
3,4b6a9c2ff964a5201ada2be3,Park,Bakery,Swiss Restaurant,Zoo Exhibit,Gym,Scenic Lookout,Bus Station,Burger Joint,French Restaurant,Gas Station
4,4b704670f964a5205e0f2de3,Bar,Burger Joint,French Restaurant,Japanese Restaurant,Pizza Place,Plaza,Department Store,Creperie,Gym,Multiplex


## Cluster Neighborhoods
Now I check the similarities of each neighbourhood with one another using k clusters 

In [21]:
# set number of clusters
kclusters = 8

lausanne_grouped_clustering = lausanne_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lausanne_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 3, 1, 5, 1, 1, 5, 1, 3], dtype=int32)

In [22]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4b5071b2f964a520742327e3,Park,Pool,Indian Restaurant,General Entertainment,Sandwich Place,Bus Station,Coffee Shop,Supermarket,Sushi Restaurant,Swiss Restaurant
1,4b56eab0f964a5202c1f28e3,Japanese Restaurant,Park,Bar,Italian Restaurant,Asian Restaurant,Bistro,Burger Joint,Nightclub,Lounge,Sushi Restaurant
2,4b5db279f964a520566729e3,Supermarket,Bus Stop,Gym / Fitness Center,Restaurant,Electronics Store,Train Station,Gym,Bowling Alley,Furniture / Home Store,Park
3,4b6a9c2ff964a5201ada2be3,Park,Bakery,Swiss Restaurant,Zoo Exhibit,Gym,Scenic Lookout,Bus Station,Burger Joint,French Restaurant,Gas Station
4,4b704670f964a5205e0f2de3,Bar,Burger Joint,French Restaurant,Japanese Restaurant,Pizza Place,Plaza,Department Store,Creperie,Gym,Multiplex
5,4b72a011f964a5200b812de3,Grocery Store,Supermarket,Italian Restaurant,Nightclub,Chinese Restaurant,Sushi Restaurant,Train Station,Bakery,Theater,Gym
6,4bc1e78fb492d13a8f94a660,Hotel,Italian Restaurant,Café,Mediterranean Restaurant,Hotel Bar,Park,French Restaurant,Restaurant,Japanese Restaurant,Cafeteria
7,4bc59590ccbcef3befaae6d2,Bar,Japanese Restaurant,Burger Joint,Pizza Place,Gym,French Restaurant,Department Store,Creperie,Plaza,Nightclub
8,4bd2e821b221c9b6ac6fd8d0,Italian Restaurant,Bar,French Restaurant,Hotel,Restaurant,Coffee Shop,Indian Restaurant,Museum,Sandwich Place,Rock Club
9,4bdb17f22a3a0f472461adb6,Supermarket,Hotel,Electronics Store,Gas Station,Shopping Mall,French Restaurant,Construction & Landscaping,Fast Food Restaurant,Restaurant,Furniture / Home Store


In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lausanne_merged = dataframe_filtered

# merge lausanne_grouped with lausanne_data to add latitude/longitude for each neighborhood
lausanne_merged = lausanne_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='id')
#dataframe_filtered
lausanne_merged.head() # check the last columns!

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Migros,Supermarket,Avenue d'Ouchy 11,CH,Lausanne,Suisse,NaN,277,"[Avenue d'Ouchy 11, 1006 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.51503921361126...",46.515039,6.632750,1006,Vaud,4bd2e821b221c9b6ac6fd8d0,1,Italian Restaurant,Bar,French Restaurant,Hotel,Restaurant,Coffee Shop,Indian Restaurant,Museum,Sandwich Place,Rock Club
1,Migros,Supermarket,Rue des Terreaux 23,CH,Lausanne,Suisse,NaN,668,"[Rue des Terreaux 23, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52248959188916...",46.522490,6.627551,1003,Vaud,4cc1bc9e43646dcb16ab478c,5,Bar,Japanese Restaurant,Burger Joint,Pizza Place,Plaza,French Restaurant,Chinese Restaurant,Department Store,Creperie,Gym
2,Migros,Grocery Store,Avenue de Rhodanie 2,CH,Lausanne,Suisse,NaN,1081,"[Avenue de Rhodanie 2, 1007 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.50773468857085...",46.507735,6.624897,1007,Vaud,4bc1e78fb492d13a8f94a660,1,Hotel,Italian Restaurant,Café,Mediterranean Restaurant,Hotel Bar,Park,French Restaurant,Restaurant,Japanese Restaurant,Cafeteria
3,Ecole Club Migros,General College & University,Rue de Genève 35,CH,Lausanne,Suisse,NaN,707,"[Rue de Genève 35, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52220032051376...",46.522200,6.625372,1003,Vaud,4c7bd61497028cfa4600e2fe,5,Bar,Japanese Restaurant,Burger Joint,Pizza Place,Nightclub,Lounge,Plaza,French Restaurant,Jewish Restaurant,Park
4,Migros,Grocery Store,Voie du Chariot 4-6,CH,Lausanne,Suisse,NaN,480,"[Voie du Chariot 4-6, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52100915635693...",46.521009,6.629060,1003,Vaud,4b704670f964a5205e0f2de3,5,Bar,Burger Joint,French Restaurant,Japanese Restaurant,Pizza Place,Plaza,Department Store,Creperie,Gym,Multiplex


In [24]:
lausanne_merged

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Migros,Supermarket,Avenue d'Ouchy 11,CH,Lausanne,Suisse,NaN,277,"[Avenue d'Ouchy 11, 1006 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.51503921361126...",46.515039,6.632750,1006,Vaud,4bd2e821b221c9b6ac6fd8d0,1,Italian Restaurant,Bar,French Restaurant,Hotel,Restaurant,Coffee Shop,Indian Restaurant,Museum,Sandwich Place,Rock Club
1,Migros,Supermarket,Rue des Terreaux 23,CH,Lausanne,Suisse,NaN,668,"[Rue des Terreaux 23, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52248959188916...",46.522490,6.627551,1003,Vaud,4cc1bc9e43646dcb16ab478c,5,Bar,Japanese Restaurant,Burger Joint,Pizza Place,Plaza,French Restaurant,Chinese Restaurant,Department Store,Creperie,Gym
2,Migros,Grocery Store,Avenue de Rhodanie 2,CH,Lausanne,Suisse,NaN,1081,"[Avenue de Rhodanie 2, 1007 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.50773468857085...",46.507735,6.624897,1007,Vaud,4bc1e78fb492d13a8f94a660,1,Hotel,Italian Restaurant,Café,Mediterranean Restaurant,Hotel Bar,Park,French Restaurant,Restaurant,Japanese Restaurant,Cafeteria
3,Ecole Club Migros,General College & University,Rue de Genève 35,CH,Lausanne,Suisse,NaN,707,"[Rue de Genève 35, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52220032051376...",46.522200,6.625372,1003,Vaud,4c7bd61497028cfa4600e2fe,5,Bar,Japanese Restaurant,Burger Joint,Pizza Place,Nightclub,Lounge,Plaza,French Restaurant,Jewish Restaurant,Park
4,Migros,Grocery Store,Voie du Chariot 4-6,CH,Lausanne,Suisse,NaN,480,"[Voie du Chariot 4-6, 1003 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52100915635693...",46.521009,6.629060,1003,Vaud,4b704670f964a5205e0f2de3,5,Bar,Burger Joint,French Restaurant,Japanese Restaurant,Pizza Place,Plaza,Department Store,Creperie,Gym,Multiplex
5,Migros,Grocery Store,Ave. de Sévelin 2A,CH,Lausanne,Suisse,NaN,1197,"[Ave. de Sévelin 2A, 1007 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52135186258868...",46.521352,6.615997,1007,Vaud,4b56eab0f964a5202c1f28e3,1,Japanese Restaurant,Park,Bar,Italian Restaurant,Asian Restaurant,Bistro,Burger Joint,Nightclub,Lounge,Sushi Restaurant
6,Migros,Grocery Store,Rue du Maupas 63,CH,Lausanne,Suisse,NaN,1146,"[Rue du Maupas 63, 1004 Lausanne, Suisse]","[{'label': 'display', 'lat': 46.52600298499138...",46.526003,6.623536,1004,Vaud,4bde7292e75c0f478ff2c703,5,Japanese Restaurant,Bar,Burger Joint,Chinese Restaurant,Brewery,Sushi Restaurant,Plaza,Pizza Place,Nightclub,Cantonese Restaurant
7,Migros,Department Store,Avenue de la Vallonnette 2,CH,Lausanne,Suisse,NaN,2004,"[Avenue de la Vallonnette 2, 1012 Lausanne, Su...","[{'label': 'display', 'lat': 46.52705916235473...",46.527059,6.651592,1012,Vaud,4bf987108d30d13a6a610218,7,Supermarket,Bakery,Sushi Restaurant,Hotel,Metro Station,Bus Station,Gas Station,Museum,Department Store,Farmers Market
8,Migros,Grocery Store,Ch. du Pré-de-la-Tour 3,CH,Pully,Suisse,NaN,2333,"[Ch. du Pré-de-la-Tour 3, 1009 Pully, Suisse]","[{'label': 'display', 'lat': 46.51126962847088...",46.511270,6.659531,1009,Vaud,4b5071b2f964a520742327e3,1,Park,Pool,Indian Restaurant,General Entertainment,Sandwich Place,Bus Station,Coffee Shop,Supermarket,Sushi Restaurant,Swiss Restaurant
9,Migros,Grocery Store,Place de la Croix-Blanche 11,CH,Epalinges,Suisse,NaN,4690,"[Place de la Croix-Blanche 11, 1066 Epalinges,...","[{'label': 'display', 'lat': 46.54840656841466...",46.548407,6.670572,1066,Vaud,4d56aab958856dcb44de5a6d,1,Bakery,Grocery Store,Italian Restaurant,Japanese Restaurant,Pub,Health & Beauty Service,Chinese Restaurant,Metro Station,Gym / Fitness Center,Zoo Exhibit


In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lausanne_merged['lat'], lausanne_merged['lng'], lausanne_merged['id'], lausanne_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Bern next!
##The exact same code was used for Bern, just the starting point was changed to the Bern train station. It would have been a waste of time to copy and paste the same code, so I recycled the prvious code used for Lausanne to find the best places in Bern. My presentation will show this and the resiults. 

## Results
Results section where you discuss the results.

## Discussion
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

## Conclusion
Conclusion section where you conclude the report.